In [59]:
import tensorflow as tf
import os
from tensorflow import keras
from keras import layers
from keras import models
import numpy as np
import pathlib

In [60]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)

PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir =  "/root/.keras/datasets/cats_and_dogs_filtered/train"
validation_dir = "/root/.keras/datasets/cats_and_dogs_filtered/validation"

In [61]:
!ls /root/.keras/datasets/cats_and_dogs_filtered/train

cats  dogs


In [62]:
train_dir = pathlib.Path(train_dir)
validation_dir = pathlib.Path(validation_dir)

In [63]:
Class_Names = np.array([item.name for item in train_dir.glob('*')])

In [64]:
Class_Names

array(['cats', 'dogs'], dtype='<U4')

In [65]:
train_dataset = tf.data.Dataset.list_files(str(train_dir/'*/*'))

In [66]:
validation_dataset = tf.data.Dataset.list_files(str(validation_dir/'*/*'))

In [67]:
def get_label(file_path):
  parts = tf.strings.split(file_path, os.path.sep)
  return parts[-2] == Class_Names

In [68]:
def load_img(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_image(img, 3, expand_animations=False)
    img = tf.cast(img, tf.float32)
    return img

In [69]:
def normalize(image):
    image = image / 255
    return image

In [70]:
def resize(image,height, width):
    image = tf.image.resize(image, (height, width),
                                 method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return image

In [71]:
def load_image_with_label(image_path):
    label = get_label(image_path)
    img = load_img(image_path)
    return img, label

In [72]:
def load_image(image_file):
    image, label = load_image_with_label(image_file)
    image = resize(image,150,150)
    image = normalize(image)
    
    return image, label

In [73]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

In [74]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
train_dataset = train_dataset.map(load_image) 
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)

In [76]:
validation_dataset = validation_dataset.map(load_image)
validation_dataset = validation_dataset.batch(BATCH_SIZE)

In [77]:
base_model = tf.keras.applications.VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

In [78]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))


In [79]:
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(lr=1e-4), metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [80]:
train_dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.bool, name=None))>

In [81]:
history = model.fit( train_dataset, epochs=30, 
                    validation_data=validation_dataset)

Epoch 1/30
63/63 [==============================] - 8s 86ms/step - loss: 0.7007 - acc: 0.5380 - val_loss: 0.6768 - val_acc: 0.5990
Epoch 2/30
63/63 [==============================] - 7s 83ms/step - loss: 0.6728 - acc: 0.5975 - val_loss: 0.6589 - val_acc: 0.6110
Epoch 3/30
63/63 [==============================] - 7s 83ms/step - loss: 0.6427 - acc: 0.6270 - val_loss: 0.6898 - val_acc: 0.5350
Epoch 4/30
63/63 [==============================] - 7s 82ms/step - loss: 0.6141 - acc: 0.6465 - val_loss: 0.6112 - val_acc: 0.6770
Epoch 5/30
63/63 [==============================] - 7s 84ms/step - loss: 0.5846 - acc: 0.6845 - val_loss: 0.5957 - val_acc: 0.6970
Epoch 6/30
63/63 [==============================] - 7s 84ms/step - loss: 0.5473 - acc: 0.7200 - val_loss: 0.6098 - val_acc: 0.6770
Epoch 7/30
63/63 [==============================] - 10s 110ms/step - loss: 0.5134 - acc: 0.7480 - val_loss: 0.5938 - val_acc: 0.6900
Epoch 8/30
63/63 [==============================] - 7s 84ms/step - loss: 0.4876 -